# ORR Thermokinetic Analysis

_______

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

# Import from Data Scripts

In [2]:
import os
import sys

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "workflow/data",
        ),
    )

from data import h2_ref
from data import h2o_ref
from data import corrections_dict

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_fe_graph"],
        "data",
        ),
    )

from proj_data_fe_graph import gas_molec_dict, scaling_dict_ideal
scaling_dict = scaling_dict_ideal

# Import Modules

In [3]:
from dft_job_automat.job_analysis import DFT_Jobs_Analysis

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

import plotly.graph_objs as go
import plotly.plotly as py

# My Modules
from orr_reaction.orr_methods import df_calc_adsorption_e
from energetics.dft_energy import Element_Refs
from orr_reaction.orr_fed_plot import ORR_Free_E_Plot

# Elemental Reference Data

In [4]:
Refs = Element_Refs()
oxy_ref = Refs.E_O_ref.gibbs_e
hyd_ref = Refs.E_H_ref.gibbs_e

Divisor is Energy class instance!!!


# Load DFT Data

In [5]:
root_dir = "/mnt/c/Users/raulf/github/Supported_Graphene_2018/02_data"


Jobs = DFT_Jobs_Analysis(
    update_job_state=False,
    job_type_class=None,
    load_dataframe=True,
    root_dir=root_dir,
    working_dir=root_dir,
    dataframe_dir=root_dir,
    )

df_m = Jobs.data_frame  # df_m -> dataframe_master -> Master Dataframe

# Calculating the Adsorption Free Energies

TEMP | For now we're just taking the spinpol=False jobs, which gives 4 jobs (bare, *ooh, *o, *oh) with energies

In [6]:
group = df_m[df_m["spinpol"] == False]

df_calc_adsorption_e(
    group,
    oxy_ref,
    hyd_ref,
    group[group["adsorbate"] == "bare"]["elec_energy"].iloc[0],
    corrections_mode="corr_dict",
#     corrections_mode=None,
#         corrections_column="gibbs_correction",
    corrections_dict=corrections_dict,
    )


# Instantiating the ORR Class

In [8]:
ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",  
    bias=0.,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="ORR",
    )

# Adding the ORR data series to the ORR instance
ORR_PLT.add_series(
    group,
    plot_mode="all",
    overpotential_type="ORR",
    )

# ** 1D ORR Limiting Potential Plot

In [10]:
from orr_reaction.orr_fed_plot import Volcano_Plot

VP = Volcano_Plot(
    ORR_PLT,
    x_ax_species="oh",
    )

VP.create_volcano_relations_plot()

volcano_legs_data = VP.create_volcano_lines(
    gas_molec_dict=gas_molec_dict,
    scaling_dict=scaling_dict,

    plot_all_legs=False,
    plot_min_max_legs=True,
    )

No smart format given!


In [11]:
layout = VP.get_plotly_layout(
    showlegend=False,
    width=1.8 * 9. * 37.795275591,
    height=1.4 * 9. * 37.795275591,
    )

save_dir = "__temp__"
py.iplot(
    {
        "data": VP.data_points + volcano_legs_data,
        "layout": layout,
        },
    filename=os.path.join(save_dir, "181101_kevin_volcano),
    )

# ** Adsorbates Scaling Relationships

In [15]:
from orr_reaction.orr_fed_plot import Scaling_Relations_Plot

SC_PLT = Scaling_Relations_Plot(
    ORR_PLT,
    plot_range={
        "y": [-2., 7.],
        "x": [-2., 3.5],
        },

    )

exclude_dict = {
    'bulk_system': 'IrO2',
    'coverage_type': 'o_covered',
#     'dopant': 'Ni',
#     'facet': '110',
#     'site': 'ir_site',
    'site': 'ir_site',
    }

# There is only a single data point, so not enough
# to do linear fits

# slope_intercept_dict = SC_PLT.fit_scaling_lines(
#     "ooh",
#     exclude_dict=exclude_dict,
#     )
# SC_PLT.add_line(slope_intercept_dict,
#     name="*OOH vs *OH Scaling",
#     color="red",
#     width=1,
#     dash="solid",
#     )


# slope_intercept_dict = SC_PLT.fit_scaling_lines(
#     "o",
#     exclude_dict=exclude_dict,
#     )
# SC_PLT.add_line(slope_intercept_dict,
#     name="*O vs *OH Scaling",
#     color="red",
#     width=1,
#     dash="solid",
#     )


SC_PLT.add_ideal_lines()


SC_PLT.create_scaling_relations_plot(
#     smart_format_dict=smart_format_dict,
    )

data_m = SC_PLT.data_points["ooh_vs_oh"] + \
    SC_PLT.data_points["o_vs_oh"] + \
    SC_PLT.data_points["oh_vs_oh"] + \
    SC_PLT.data_lines

# layout = SC_PLT.layout

layout = SC_PLT.__create_layout__(
    x_ax_spec="oh",
    title="Scaling Relations",
    showlegend=False,
    )
layout["width"] = 0.8 * 18.7 * 37.795275591
layout["height"] = 0.8 * 18.7 * 37.795275591

fig = dict(data=data_m, layout=layout)
# fig = dict(data=data_ooh_oh, layout=layout_ooh_oh)
py.iplot(fig, filename=os.path.join(save_dir, "181101_kevin_scaling"))



No smart format given!


# **ORR Free Energy Diagrams

In [17]:
layout = ORR_PLT.plotly_fed_layout(
    plot_title="",
    plot_width=1.5 * 680,
    plot_height=2.5 * 510,
    )

py.plotly.iplot(
    {
        "data": ORR_PLT.plotly_data(),
        "layout": layout,
        },
        filename=os.path.join(save_dir, "181101_kevin_fed"),
    )


In [12]:
col_list = ['Job',
 'adsorbate',
 'graphene_layers',
 'max_revision',
 'path',
 'revision_number',
 'stacking',
 'support_bulk_str',
 'support_elem',
 'support_facet',
 'elec_energy',
 'init_atoms',
 'atoms_object',
 'beefensemble',
 'convergence',
 'dipole',
 'dw',
 'kpts',
 'nbands',
 'noncollinear',
 'outdir',
 'output',
 'parflags',
 'printensemble',
 'pw',
 'sigma',
 'spinpol',
 'xc']

In [13]:
# Jobs = DFT_Jobs_Analysis(
    
#     tree_level=None,
#     level_entries=None,
#     skip_dirs_lst=None,
#     indiv_dir_lst=None,  # <-----------------------------------------------
#     indiv_job_lst=None,
#     root_dir=".",
#     working_dir=".",
#     update_job_state=False,
#     load_dataframe=True,
#     dataframe_dir=None,
#     job_type_class=None,
#     methods_to_run=None,
#     folders_exist=None,

# #     indiv_dir_lst=dir_list,
# #     working_dir=".",
# #     folders_exist=True,
# #     load_dataframe=False,
# #     job_type_class=dft_inst,
#     )


# from os.path import join as join
# import sys
# sys.path.append(".")

# import pandas as pd
# from methods import job_maint

# from dft_job_automat.job_types_classes.dft_methods import DFT_Methods